# Download the dataset

# Import required Library

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

# Read dataset and do pre-processing

In [11]:
# read dataset
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
# dropping unwanted columns
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [13]:
# Count of Spam and Ham values
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [14]:
# Label Encoding target column
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [15]:
# Test and train split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [18]:
# Tokenization function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

# Create model

## Add Layers(LSTM, Dense-(Hidden Layers), Output)

In [19]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

# Compile the model

In [21]:
model = Model(inputs=inputs,outputs=layer)
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

# Fit the model

In [23]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 8s 264ms/step - loss: 0.6043 - accuracy: 0.9432 - val_loss: 0.1640 - val_accuracy: 0.9715
Epoch 2/10
30/30 [==============================] - 8s 260ms/step - loss: 0.0199 - accuracy: 0.9934 - val_loss: 0.1304 - val_accuracy: 0.9778
Epoch 3/10
30/30 [==============================] - 8s 258ms/step - loss: 0.0062 - accuracy: 0.9979 - val_loss: 0.1227 - val_accuracy: 0.9810
Epoch 4/10
30/30 [==============================] - 13s 454ms/step - loss: 0.0052 - accuracy: 0.9982 - val_loss: 0.1351 - val_accuracy: 0.9831
Epoch 5/10
30/30 [==============================] - 8s 260ms/step - loss: 0.0070 - accuracy: 0.9984 - val_loss: 0.1214 - val_accuracy: 0.9789
Epoch 6/10
30/30 [==============================] - 10s 321ms/step - loss: 0.0037 - accuracy: 0.9992 - val_loss: 0.1451 - val_accuracy: 0.9810
Epoch 7/10
30/30 [==============================] - 8s 262ms/step - loss: 0.0048 - accuracy: 0.9987 - val_loss: 0.1310 - val_accuracy: 0.9810
Epoc

# Save the model

In [24]:
model.save('model.1')

# Test the model

In [25]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [26]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 22ms/step - loss: 0.1511 - accuracy: 0.9797
Accuracy: 0.980


In [29]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(4))

27/27 [==============================] - 1s 22ms/step
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]]


In [31]:
print(Y_test[25:40],round(2))

[[0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]] 2
